In [41]:
import xml.etree.ElementTree as ET
import xmljson
import pandas as pd
import pyodbc
import os
from pprint import pprint
import xmltodict

pd.set_option('max_colwidth', 160)

conn_str = r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};"\
    r"DBQ=C:\Users\gdoku\YandexDisk\Документы\Семейные финансы\FF.accdb;"

def read_vtb_broker_reports(path):
    """Считываем данные по движению денежных средств из брокерского отчёта Банка ВТБ в формате xml"""
    
    directory = r'C:/Users/gdoku/YandexDisk/Документы/Договоры/Банк ВТБ (ПАО)/'
    files = os.listdir(os.path.join(directory, path))
    counter = 0   
    
    # Считываем данные из файлов
    for file in files:
        tree = ET.parse(os.path.join(directory, path, file))    
        root = tree.getroot()
        for i in range(2):
            json = xmljson.gdata.data(root[11][0][i][0])
            json = json['{report577p_v1}Подробности16_Collection']['{report577p_v1}Подробности16']
            data = pd.DataFrame(json)
            if counter == 0:
                transactions = data
                counter += 1
            else:
                transactions = transactions.append(data, ignore_index=True) 
                
    # Обработка данных
    transactions.columns = ['date', 'amount', 'currency', 'description', 'notes']
    transactions = transactions[transactions.currency == 'RUR']
    transactions.date = pd.to_datetime(transactions.date, format='%Y-%m-%d')
    transactions.description = transactions.description + '; ' + transactions.notes
    transactions = transactions[['date', 'amount', 'description']]
    transactions = transactions.sort_values(by=['date'])

    return transactions

path = r'Брокерские отчёты XML/'
transactions = read_vtb_broker_reports(path)
transactions

,date,amount,description
0,2020-01-02,-445.16,Вознаграждение Брокера; Разница между суммами по специальным сделкам РЕПО ¹
1,2020-01-02,-327.96,Вознаграждение Брокера; Комиссия банка за Спецсделки РЕПО
2,2020-01-02,329.83,Сальдо расчётов по сделкам с ценными бумагами;
5,2020-01-03,-1336.27,Вознаграждение Брокера; Разница между суммами по специальным сделкам РЕПО ¹
833,2020-01-03,31850.00,Вариационная маржа;
...,...,...,...
1197,2020-09-03,-6.50,Вознаграждение Брокера; Комиссия за брокерские услуги по проведению расчетов по заключенным сделкам на Срочном рынке
831,2020-09-04,-936.70,Сальдо расчётов по сделкам с ценными бумагами;
830,2020-09-04,-275.65,Вознаграждение Брокера; Комиссия банка за Спецсделки РЕПО
832,2020-09-04,-244.77,Вознаграждение Брокера; Разница между суммами по специальным сделкам РЕПО ¹


In [2]:
transactions[11][0][0][0][0].attrib

{'debt_type4': '2020-01-02T00:00:00',
 'debt_date4': '-445.16',
 'decree_amount2': 'RUR',
 'operation_type': 'Вознаграждение Брокера',
 'notes1': 'Разница между суммами по специальным сделкам РЕПО ¹'}

In [3]:
dict(transactions[11][0][0][0][0].attrib)

{'debt_type4': '2020-01-02T00:00:00',
 'debt_date4': '-445.16',
 'decree_amount2': 'RUR',
 'operation_type': 'Вознаграждение Брокера',
 'notes1': 'Разница между суммами по специальным сделкам РЕПО ¹'}

In [4]:
o = xmltodict.parse(transactions[11][0][0][0])
o

TypeError: a bytes-like object is required, not 'xml.etree.ElementTree.Element'

In [5]:
transactions[11][0][0][0][0].attrib
json = xmljson.gdata.data(transactions[11][0][1][0])
json = json['{report577p_v1}Подробности16_Collection']['{report577p_v1}Подробности16']
json

[OrderedDict([('debt_type4', '2020-01-03T00:00:00'),
              ('debt_date4', 31850.0),
              ('decree_amount2', 'RUR'),
              ('operation_type', 'Вариационная маржа'),
              ('notes1', '')]),
 OrderedDict([('debt_type4', '2020-01-06T00:00:00'),
              ('debt_date4', 5850.0),
              ('decree_amount2', 'RUR'),
              ('operation_type', 'Вариационная маржа'),
              ('notes1', '')]),
 OrderedDict([('debt_type4', '2020-01-08T00:00:00'),
              ('debt_date4', 40950.0),
              ('decree_amount2', 'RUR'),
              ('operation_type', 'Вариационная маржа'),
              ('notes1', '')]),
 OrderedDict([('debt_type4', '2020-01-09T00:00:00'),
              ('debt_date4', -50000.0),
              ('decree_amount2', 'RUR'),
              ('operation_type', 'Списание денежных средств')]),
 OrderedDict([('debt_type4', '2020-01-09T00:00:00'),
              ('debt_date4', -20000.0),
              ('decree_amount2', 'RUR'),
     

In [6]:
df = pd.DataFrame(json)

In [7]:
df

,debt_type4,debt_date4,decree_amount2,operation_type,notes1
0,2020-01-03T00:00:00,31850.0,RUR,Вариационная маржа,
1,2020-01-06T00:00:00,5850.0,RUR,Вариационная маржа,
2,2020-01-08T00:00:00,40950.0,RUR,Вариационная маржа,
3,2020-01-09T00:00:00,-50000.0,RUR,Списание денежных средств,NaN
4,2020-01-09T00:00:00,-20000.0,RUR,Списание денежных средств,NaN
...,...,...,...,...,...
361,2020-09-02T00:00:00,-14880.0,RUR,Вариационная маржа,
362,2020-09-03T00:00:00,-18445.0,RUR,Вариационная маржа,
363,2020-09-03T00:00:00,-1.0,RUR,Вознаграждение Брокера,Комиссия банка за заключение сделок на Срочном рынке
364,2020-09-03T00:00:00,-6.5,RUR,Вознаграждение Брокера,Комиссия за брокерские услуги по проведению расчетов по заключенным сделкам на Срочном рынке


In [153]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 833 entries, 0 to 832
Data columns (total 5 columns):
debt_type4        833 non-null object
debt_date4        833 non-null float64
decree_amount2    833 non-null object
operation_type    833 non-null object
notes1            689 non-null object
dtypes: float64(1), object(4)
memory usage: 32.7+ KB


In [159]:
set(df['operation_type'])

{'Вариационная маржа',
 'Вознаграждение Брокера',
 'Зачисление денежных средств',
 'НДФЛ',
 'Списание денежных средств'}

In [163]:
df[df.operation_type == 'Вознаграждение Брокера']

,debt_type4,debt_date4,decree_amount2,operation_type,notes1
39,2020-02-13T00:00:00,-1.00,RUR,Вознаграждение Брокера,Комиссия банка за заключение сделок на Срочном рынке
40,2020-02-13T00:00:00,-6.89,RUR,Вознаграждение Брокера,Комиссия за брокерские услуги по проведению расчетов по заключенным сделкам на Срочном рынке
59,2020-02-26T00:00:00,-1.00,RUR,Вознаграждение Брокера,Комиссия банка за заключение сделок на Срочном рынке
60,2020-02-26T00:00:00,-6.64,RUR,Вознаграждение Брокера,Комиссия за брокерские услуги по проведению расчетов по заключенным сделкам на Срочном рынке
67,2020-02-28T00:00:00,-9.00,RUR,Вознаграждение Брокера,Комиссия банка за заключение сделок на Срочном рынке
68,2020-02-28T00:00:00,-32.20,RUR,Вознаграждение Брокера,Комиссия за брокерские услуги по проведению расчетов по заключенным сделкам на Срочном рынке
70,2020-03-02T00:00:00,-2.00,RUR,Вознаграждение Брокера,Комиссия банка за заключение сделок на Срочном рынке
71,2020-03-02T00:00:00,-12.24,RUR,Вознаграждение Брокера,Комиссия за брокерские услуги по проведению расчетов по заключенным сделкам на Срочном рынке
79,2020-03-10T00:00:00,-0.06,RUR,Вознаграждение Брокера,Комиссия банка за заключение сделок на Срочном рынке
80,2020-03-10T00:00:00,-0.06,RUR,Вознаграждение Брокера,Комиссия за брокерские услуги по проведению расчетов по заключенным сделкам на Срочном рынке
